In [171]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Portfolio 2 - The Rescorla-Wagner Model

In this portfolio, we are going to implement the Rescorla-Wagner model to simulate the behavior of agents interacting with environments that can have different levels of volatility. The Rescorla-Wagner model is a model of classical conditioning in which learning occurs in response to a prediction error. The model itself is a simple algebraic equation describing how associative values between a stimulus and an outcome are updated via error correction. 

This exercise will also introduce you to concepts like *learning rate*, *prediction error*, *volatility*, and *uncertainty* that are central in the reinforcement learning literature and computational psychiatry.

It is recommended that you first explore some resources to get a good intuition of the model before moving to the implementation part. The following videos are good introduction to the core concepts: [video-1](https://www.youtube.com/watch?v=D8b-cflPpec), [video-2](https://www.youtube.com/watch?v=CXrMtA1eNvQ).

**Additional resources**

> [Wikipedia](https://en.wikipedia.org/wiki/Rescorla%E2%80%93Wagner_model)

> [Scholarpedia](http://www.scholarpedia.org/article/Rescorla-Wagner_model)

> Sutton, R. & Barto, A. (2018). Reinforcement learning: an introduction. Cambridge, Massachusetts London, England: The MIT Press. *14.2.2. The Rescorla-Wagner Model*.

* [BayesCog Summer 2020 Lecture 09 - Intro to cognitive modeling & Rescorla-Wagner model](https://www.youtube.com/watch?v=tXFKYWx6c3k)
* [BayesCog Summer 2020 Lecture 10 - Implementing Rescorla-Wagner in Stan](https://www.youtube.com/watch?v=M69theIxI3g)

## Learning rate

In this exercise, we will create a simple version of the Rescorla-Wagner model that can update the associative value of a stimulus based on the prediction error of the outcome. We would like to reproduce the following figure from [Pulcu & Browning (2019)](https://doi.org/10.1016/j.tics.2019.07.007): 

![title](https://marlin-prod.literatumonline.com/cms/attachment/cdba6dca-87e1-480b-b7ee-d6f554ba8a72/gr1.jpg)

The core idea here is that task volatility (the rapid change of probability outcomes associated with conditioned stimuli) requires an adaptation of the learning rate of a basic (Rescorla-Wagner) reinforcement learning model. You are going to demonstrate that by simulating the performances of 2 agents (high and low learning rate) performing 2 tasks (high and low cat volatility).

Our agent cannot tell beforehand if the cat will scratch him or not, but he can learn from previous experience and update his expectation that he will be scratched in the future. Two key parameters are guiding the value update: the *prediction error* and the *learning rate*.

The *prediction error* is parameterized as:

$$ PE_{t-1} = R_{t-1} - V_{t-1} $$

Updating the value function is defined by:

$$ V_{t} = V_{t-1} + \alpha * PE $$

where $R$ is the outcome, $V$ is the associative strength between the CS and the outcome, and $\alpha$ is the learning rate.

## Exercise

You should create a Python class named `RescorlaWagner`, this class should have 3 core methods: 
* `response()` will generate the agent decision (e.g. stroke the cat or not).
* `update()` will update the outcome (did the cat scratch the agent or not, update the prediction for the next trial).
* `plot()` will plot the result from the entire experiment (hidden probability, responses, and associative value).

The final plot might look like something like this (this is just for illustration, the final traces can differ):
![title](https://github.com/LegrandNico/CognitiveModeling/raw/master/notebooks/data/learningRate.png)

All data and relevant parameters should be stored in the class attributes. All methods should be documented following the Matplotlib standard (see [here](https://matplotlib.org/stable/devel/documenting_mpl.html)). You cannot import any additional package (just use Numpy and Matplotlib).

The two cat volatility levels are given by the following vectors:

In [ ]:
volatileCat = np.repeat(
    np.array([0.5, 0.8, 0.1, 0.8, 0.1, 0.8, 0.1, 0.8, 0.1, 0.8]),
    10, axis=0)

stableCat = np.repeat(
    np.array([0.5, 0.8, 0.1, 0.8]),
    25, axis=0)

1) You should use two agents and make them learn the underlying associations using a high and a low learning rate (e.g 0.2 vs 0.8). You should have four plots in total (learning rate * task variability).

2) How can we find an optimal learning rate that would maximize the outcome (stroke the cat without being scratched most of the time)? Run 1000 simulations and use a robust estimate.

## Blocking

We have built from the previous exercise an agent that can update its expectation about future outcomes associated with a stimulus. However, one of the main strengths of the Rescorla-Wagner model, and the reason why it is still used as a baseline reference for a reinforcement learning task, is that it can explain results that occur under more complex setups like blocking.

Blocking happens when many CSs are presented in association with an outcome, but only some of them show a conditioning response. In the previous example, we used only one CS (the cat) that was always presented, and the outcome probability was varying through time. Here, we are going to present 2 stimuli ($A$ and $X$).

$V_{A}$, $V_{X}$ and $V_{AX}$ denote the associative strength of stimuli $A$, $X$ and the compound $AX$. Suppose that on a trial the compound $CS_{AX}$ is followed by a $US$ laballed stimulus $Y$. Then the associative strengths of the stimulus components change according to these expression:

$$ \Delta V_{A} = \alpha_{A}\beta_{Y}(R_{Y}-V_{AX})$$
$$ \Delta V_{X} = \alpha_{X}\beta_{Y}(R_{Y}-V_{AX})$$

Here, $V_{AX} = V_{A} + V_{X}$. $\alpha$ refers to the $CS$ salience and $\beta$ refers to the $US$ salience, which can be 0 or 1 given the array below. $\alpha_{a}$ is encoded in `stimulusA`, $\alpha_{x}$ is encoded in `stimulusX` and $\beta_{Y}$ is encoded on `unconditioned`. $R_{Y}$ is the asymptotic level of associative strength that the US $Y$ can support. In this case, this will be set to 1. Note that we are not explicitely describing the learning rate here, but this can be added by scaling $\Delta V_{A}$ and $\Delta V_{X}$ by some number between 0 and 1.

In [165]:
unconditioned = np.repeat(
    np.array([0, 1, 0, 1, 0, 0]),
    10, axis=0)

stimulusA = np.repeat(
    np.array([0, 1, 0, 1, 0, 0]),
    10, axis=0)

stimulusX = np.repeat(
    np.array([0, 0, 0, 1, 0, 1]),
    10, axis=0)

## Exercise

* Using the previous `RescorlaWagner` class, make some modifications that will let it use a compound of CS variables (here $A$ and $X$) to predict the unconditioned stimulus. Because the association $A-Y$ is learned first, we should observe a *blocking* of $X-Y$ when it is later presented.

* Create a figure synthesizing the main result from this task, demonstrating that the Rescorla-Wagner model can explain the blocking effect. Ideally, the figure should have 5 rows:
* Unconditioned stimulus $Y$
* Stimulus $A$
* Stimulus $X$
* $V_{A}$
* $V_{X}$